## With a convex hull; 2D

In [5]:
def visualize2DPCA_hull(matrix, membershipVector, membFiltered, colors, selector = '', annotate = False):
    '''The function produces a 2D scatter plot of a given matrix;
    it requires a membership vector which will be used for filtering and coloring groups of points differently
    
    the 'colors' dictionary needs to have the colors for each of the groups present in the membership vector'''

    from matplotlib import pyplot as plt
    from scipy.spatial import ConvexHull
    import numpy as np
    
    fig = plt.figure(figsize=(25, 25))
    ax = fig.add_subplot(111)
    ax.set_xticklabels(labels=['']*len(membershipVector), alpha=0)
    ax.set_yticklabels(labels=['']*len(membershipVector), alpha=0)
    ax.tick_params(length=10)
    
    if not selector:
        selector = colors
    
    for each in selector:
        
        # to subset the rows for each cohort in order to easily label them on the graph
        tmpIndex = [i for i,m in enumerate(membershipVector) if m == each]        

        x = [coord for i, coord in enumerate(matrix[:, 0]) if i in tmpIndex]
        y = [coord for i, coord in enumerate(matrix[:, 1]) if i in tmpIndex]

        plt.scatter(x, y, c=colors[each], s=[100], label = each, edgecolors='black', linewidths=0.3)
        
        #Draw the convex hull
        
        points = np.column_stack((x, y))
        
        #must have at least two points for a hull
        if len(points) > 2:
            
            hull = ConvexHull(points)      

            for simplex in hull.simplices:
                
#                 print(simplex)

                plt.plot(points[simplex, 0], points[simplex, 1], '--', c=colors[each], alpha = 0)
        
            plt.fill(points[hull.vertices,0], points[hull.vertices,1], c=colors[each], alpha=0.1)

    # annotate
            
    if annotate:
        for i, txt in enumerate([f'{c}' for c in membFiltered]):
            
            ax.annotate(txt, (matrix[i, 0], matrix[i, 1]))


    plt.legend(prop = {'size': 10})
    plt.show()

## With hulls; 3D

In [4]:
def visualize3DData_hull(matrix, membershipVector, annotationIndex, colors, selector = ''):
    """Visualize data in 3d scatterplot with a convex hull around each group.

    Args:
        X (np.array) - array of points, of shape (numPoints, 3)
        (likely a PCA dimensionality reduction dataframe)
        
        membershipVector (list) - a list containing the group membership of each point (row)
                
        annotationIndex (list) - a list containing the desired labels of each point (row)
        
        colors (dict) - a dictionary specifying the color choice of each group within the memberhsip vector
        
    Returns:
        None; plots the 3d scatterplot
    """
    
    from matplotlib import pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib import pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from matplotlib.colors import to_rgba
    from scipy.spatial import ConvexHull
    import numpy as np
    
    fig = plt.figure(figsize = (32,20))
    ax = fig.add_subplot(111, projection = '3d')
    ax = Axes3D(fig)
    ax.set_xticklabels(labels='')
    ax.set_yticklabels(labels='')
    ax.set_zticklabels(labels='', ticklabels='')
    
    if not selector:
        selector = colors
    
    for each in selector:

        # to subset the rows for each cohort in order to easily label them on the graph
        tmpIndex = [i for i,m in enumerate(membershipVector) if m == each]        

        x = [coord for i, coord in enumerate(matrix[:, 0]) if i in tmpIndex]
        y = [coord for i, coord in enumerate(matrix[:, 1]) if i in tmpIndex]
        z = [coord for i, coord in enumerate(matrix[:, 2]) if i in tmpIndex]

        
        ax.scatter(x, y, z,
                   c=[colors[each]] * len(x), s = [200],
                   depthshade = False, picker = True, label = each, edgecolors='black', linewidths=0.3)


        #Draw the convex hull
        
        if len(x) > 4:
            
            pts = np.column_stack((x, y, z))
            hull = ConvexHull(pts)

            for s in hull.simplices:
                s = np.append(s, s[0])  # Here we cycle back to the first coordinate
                
                #uncomment bellow if you want to plot the lines too
#                 ax.plot(pts[s, 0], pts[s, 1], pts[s, 2], c = colors[each], alpha = .1)                

            #get the vertices from the convex hull
            indices = hull.simplices
            vertices = pts[indices]
        
            #add a polygon connecting the vertices
            collection = Poly3DCollection(vertices, alpha=0.15)            
            face_color = to_rgba(colors[each])
            collection.set_facecolor(face_color)
            ax.add_collection3d(collection)
                            
    ax.legend(prop={'size': 15})
    annIndex = annotationIndex

# 3d with hulls; only hulls except for pigu

In [3]:
def visualize3DData_hull2(X, membershipVector, annotationIndex, colors, selector = ''):
    """ONLY FOR PIGU ANALYSIS
    
    Visualize data in 3d scatterplot with a convex hull around each group.

    Args:
        X (np.array) - array of points, of shape (numPoints, 3)
        (likely a PCA dimensionality reduction dataframe)
        
        membershipVector (list) - a list containing the group membership of each point (row)
                
        annotationIndex (list) - a list containing the desired labels of each point (row)
        
        colors (dict) - a dictionary specifying the color choice of each group within the memberhsip vector
        
    Returns:
        None; plots the 3d scatterplot
    """
    
    from matplotlib import pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D
    from matplotlib import pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Poly3DCollection
    from matplotlib.colors import to_rgba
    from scipy.spatial import ConvexHull
    import numpy as np
    
    fig = plt.figure(figsize = (32,20))
    ax = fig.add_subplot(111, projection = '3d')
    ax = Axes3D(fig)
    ax.set_xticklabels(labels='')
    ax.set_yticklabels(labels='')
    ax.set_zticklabels(labels='', ticklabels='')
    
    if not selector:
        selector = colors
    
    for each in selector:

        # to subset the rows for each cohort in order to easily label them on the graph
        tmpIndex = [index for index in range(len(membershipVector)) if membershipVector[index] == each]        
        currentSubset = np.array(tmpIndex, dtype=np.int8)
        
        if each == 'PIGU':
            ax.scatter(X[currentSubset[:, None], 0], X[currentSubset[:, None], 1], X[currentSubset[:, None], 2],
                       c=[colors[each]] * len(currentSubset), s = [200],
                       depthshade = False, picker = True, label = each, edgecolors='black', linewidths=0.3)

        x, y, z = X[currentSubset[:, None], 0], X[currentSubset[:, None], 1], X[currentSubset[:, None], 2]

        #Draw the convex hull
        
        if len(x) > 4:
            
            pts = np.column_stack((x, y, z))
            hull = ConvexHull(pts)

            for s in hull.simplices:
                s = np.append(s, s[0])  # Here we cycle back to the first coordinate
                
                #uncomment bellow if you want to plot the lines too
#                 ax.plot(pts[s, 0], pts[s, 1], pts[s, 2], c = colors[each], alpha = .1)                

            #get the vertices from the convex hull
            indices = hull.simplices
            vertices = pts[indices]
        
            #add a polygon connecting the vertices
            collection = Poly3DCollection(vertices, alpha=0.15)            
            face_color = to_rgba(colors[each])
            collection.set_facecolor(face_color)
            ax.add_collection3d(collection)
                            
    ax.legend(prop={'size': 15})
    annIndex = annotationIndex